# `Geoload`

In [7]:
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

# https://py4e-data.dr-chuck.net/opengeo?q=Ann+Arbor%2C+MI
serviceurl = 'https://py4e-data.dr-chuck.net/opengeo?'

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('opengeo.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open("where.data")
count = 0
nofound = 0
for line in fh:
    if count > 100 :
        print('Retrieved 100 locations, restart to retrieve more')
        break

    address = line.strip()
    if address == 'Viaduto Otávio Rocha':
        print('[\033[32mINFO\033[0m] Pesquisando Viaduto Otávio Rocha')
        
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database", address)
        continue
    except:
        pass

    parms = dict()
    parms['q'] = address

    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if not js or 'features' not in js:
        print('==== Download error ===')
        print(data)
        break

    if len(js['features']) == 0:
        print('==== Object not found ====')
        nofound = nofound + 1

    cur.execute('''INSERT INTO Locations (address, geodata)
        VALUES ( ?, ? )''',
        (memoryview(address.encode()), memoryview(data.encode()) ) )

    conn.commit()

    if count % 10 == 0 :
        print('Pausing for a bit...')
        #time.sleep(3)

if nofound > 0:
    print('Number of features for which the location could not be found:', nofound)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")




Found in database AGH University of Science and Technology

Found in database Academy of Fine Arts Warsaw Poland

Found in database American University in Cairo

Found in database Arizona State University

Found in database Athens Information Technology

Found in database BITS Pilani

Found in database Babcock University

Found in database Banaras Hindu University

Found in database Bangalore University

Found in database Baylor University

Found in database Beijing normal university

Found in database Belarusian State University

Found in database Belgrade University

Found in database Beloit College

Found in database Belorussian State University

Found in database Ben Gurion University

Found in database Boston University

Found in database California Polytechnic State University of San Luis Obispo

Found in database California State University San Bernardino

Found in database City of Westminster College

Found in database Columbia University

Found in database Cranfield Universit

# `Geodump`

In [8]:
import sqlite3
import json
import codecs

conn = sqlite3.connect('opengeo.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
fhand = codecs.open('where.js', 'w', "utf-8")
fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1].decode())
    try: js = json.loads(str(data))
    except: continue

    if len(js['features']) == 0: continue

    try:
        lat = js['features'][0]['geometry']['coordinates'][1]
        lng = js['features'][0]['geometry']['coordinates'][0]
        where = js['features'][0]['properties']['display_name']
        where = where.replace("'", "")
    except:
        print('Unexpected format')
        print(js)

    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print(count, "records written to where.js")
print("Open where.html to view the data in a browser")



AGH University of Science and Technology 50.065703299999996 19.918958667058632
Academy of Fine Arts in Warsaw 52.2394966 21.015693129786936
American University in Cairo - AUC 30.020531 31.5014894
Arizona State University 33.4213174 -111.93315932269833
Athens Information Technology 37.9425172 23.8706149
BITS Pilani 17.547185 78.572791
Babcock University 6.88997525 3.7224790878605756
Banaras Hindu University 25.2662887 82.9927969
Bangalore University 12.944172049999999 77.5091485894286
Baylor University 31.55043405 -97.1102905567766
Beijing Normal University (BNU) 39.959732 116.35973697755406
Belarusian State University 53.83806745 27.476155257195504
Belgrade University 44.81842 20.45757
Beloit College 42.5009819 -89.0157777411969
Unexpected format
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-90.45997, 35.95422]}, 'properties': {'country_code': 'us', 'country': 'United States', 'county': 'Craighead County', 'datasource': {'